Определим декоратор для измерения времени асинхронных функций

In [1]:
import functools
import time
from typing import Callable, Any


def async_timed():
    def wrapper(func: Callable) -> Callable:
        @functools.wraps(func)
        async def wrapped(*args, **kwargs) -> Any:
            print(f'starting {func} with args {args} {kwargs}')
            start = time.time()
            try:
                return await func(*args, **kwargs)
            finally:
                end = time.time()
                total = end - start
                print(f'finished {func} in {total:.4f} second(s)')

        return wrapped

    return wrapper

определим ключевую асинхронную функцию, которая позволяет выполнять код параллельно 

In [3]:
import asyncio

@async_timed()
async def delay(delay_seconds: int) -> int:
    print(f'sleeping for {delay_seconds} second(s)')
    await asyncio.sleep(delay_seconds)
    print(f'finished sleeping for {delay_seconds} second(s)')
    return delay_seconds

In [5]:
@async_timed()
async def test_01():
    first_delay_call = await delay(2)
    second_delay_call = await delay(3)

In [8]:
await test_01()

starting <function test_01 at 0x0000022B1DEA1440> with args () {}
starting <function delay at 0x0000022B1DCFD120> with args (2,) {}
sleeping for 2 second(s)
finished sleeping for 2 second(s)
finished <function delay at 0x0000022B1DCFD120> in 2.0056 second(s)
starting <function delay at 0x0000022B1DCFD120> with args (3,) {}
sleeping for 3 second(s)
finished sleeping for 3 second(s)
finished <function delay at 0x0000022B1DCFD120> in 3.0023 second(s)
finished <function test_01 at 0x0000022B1DEA1440> in 5.0079 second(s)


In [9]:
@async_timed()
async def test_02():
    first_task = asyncio.create_task(delay(2))
    second_task = asyncio.create_task(delay(3))
    
    await first_task
    await second_task

In [10]:
await test_02()

starting <function test_02 at 0x0000022B1E358360> with args () {}
starting <function delay at 0x0000022B1DCFD120> with args (2,) {}
sleeping for 2 second(s)
starting <function delay at 0x0000022B1DCFD120> with args (3,) {}
sleeping for 3 second(s)
finished sleeping for 2 second(s)
finished <function delay at 0x0000022B1DCFD120> in 2.0094 second(s)
finished sleeping for 3 second(s)
finished <function delay at 0x0000022B1DCFD120> in 2.9997 second(s)
finished <function test_02 at 0x0000022B1E358360> in 2.9997 second(s)


Рассмотрим вычислительно сложные задачи

In [ ]:
@async_timed()
async def cpu_bound_work(number: int = 100000000) -> int:
    counter = 0
    for i in range(number):
        counter = counter + 1
    return counter

In [12]:
await cpu_bound_work()

starting <function cpu_bound_work at 0x0000022B1E358900> with args () {}
finished <function cpu_bound_work at 0x0000022B1E358900> in 2.3243 second(s)


100000000

In [13]:
@async_timed()
async def test_03():
    task_one = asyncio.create_task(cpu_bound_work())
    task_two = asyncio.create_task(cpu_bound_work())
    await task_one
    await task_two

In [14]:
await test_03()

starting <function test_03 at 0x0000022B1E359300> with args () {}
starting <function cpu_bound_work at 0x0000022B1E358900> with args () {}
finished <function cpu_bound_work at 0x0000022B1E358900> in 2.2699 second(s)
starting <function cpu_bound_work at 0x0000022B1E358900> with args () {}
finished <function cpu_bound_work at 0x0000022B1E358900> in 2.2943 second(s)
finished <function test_03 at 0x0000022B1E359300> in 4.5642 second(s)


In [15]:
@async_timed()
async def test_04():
    task_one = asyncio.create_task(cpu_bound_work())
    task_two = asyncio.create_task(cpu_bound_work())
    delay_task = asyncio.create_task(delay(2))
    await task_one
    await task_two
    await delay_task

In [16]:
await test_04()

starting <function test_04 at 0x0000022B1E359440> with args () {}
starting <function cpu_bound_work at 0x0000022B1E358900> with args () {}
finished <function cpu_bound_work at 0x0000022B1E358900> in 2.3157 second(s)
starting <function cpu_bound_work at 0x0000022B1E358900> with args () {}
finished <function cpu_bound_work at 0x0000022B1E358900> in 2.2628 second(s)
starting <function delay at 0x0000022B1DCFD120> with args (2,) {}
sleeping for 2 second(s)
finished sleeping for 2 second(s)
finished <function delay at 0x0000022B1DCFD120> in 2.0018 second(s)
finished <function test_04 at 0x0000022B1E359440> in 6.5813 second(s)


In [17]:
@async_timed()
async def test_05():
    delay_task = asyncio.create_task(delay(2))
    task_one = asyncio.create_task(cpu_bound_work())
    task_two = asyncio.create_task(cpu_bound_work())
    await delay_task
    await task_one
    await task_two

In [18]:
await test_05()

starting <function test_05 at 0x0000022B1E359A80> with args () {}
starting <function delay at 0x0000022B1DCFD120> with args (2,) {}
sleeping for 2 second(s)
starting <function cpu_bound_work at 0x0000022B1E358900> with args () {}
finished <function cpu_bound_work at 0x0000022B1E358900> in 2.2425 second(s)
starting <function cpu_bound_work at 0x0000022B1E358900> with args () {}
finished <function cpu_bound_work at 0x0000022B1E358900> in 2.2532 second(s)
finished sleeping for 2 second(s)
finished <function delay at 0x0000022B1DCFD120> in 4.4957 second(s)
finished <function test_05 at 0x0000022B1E359A80> in 4.4957 second(s)


In [19]:
@async_timed()
async def test_06():
    task_one = asyncio.create_task(cpu_bound_work())
    delay_task = asyncio.create_task(delay(2))
    task_two = asyncio.create_task(cpu_bound_work())
    await task_one
    await delay_task
    await task_two

In [20]:
await test_06()

starting <function test_06 at 0x0000022B1E35A8E0> with args () {}
starting <function cpu_bound_work at 0x0000022B1E358900> with args () {}
finished <function cpu_bound_work at 0x0000022B1E358900> in 2.3133 second(s)
starting <function delay at 0x0000022B1DCFD120> with args (2,) {}
sleeping for 2 second(s)
starting <function cpu_bound_work at 0x0000022B1E358900> with args () {}
finished <function cpu_bound_work at 0x0000022B1E358900> in 2.2438 second(s)
finished sleeping for 2 second(s)
finished <function delay at 0x0000022B1DCFD120> in 2.2438 second(s)
finished <function test_06 at 0x0000022B1E35A8E0> in 4.5571 second(s)
